<a href="https://colab.research.google.com/github/sharvariK-11/NST-/blob/main/Chapter3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Chapter 3


downloading the datasets


In [ ]:
!pip install transformers
import transformers
!pip install transformers[sentencepiece]


In [ ]:
!pip3 install datasets


In [ ]:
from datasets import load_dataset
raw_datasets = load_dataset("glue", "mrpc")


In [ ]:
raw_train_dataset=raw_datasets["train"]
raw_train_dataset[0]


In [ ]:
raw_datasets["train"][0]

In [ ]:
raw_datasets["train"][6]
raw_datasets["train"][:5]


In [ ]:
raw_train_dataset.features

In [ ]:
raw_datasets["train"].features

tokenise and apply functiomn over all split of dataset


In [ ]:
from transformers import AutoTokenizer

checkpoint="bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
  return tokenizer(
      example["sentence1"], example["sentence2"], padding="max_length", truncation=True, max_length=128
  )
 
tokenized_datasets=raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets=tokenized_datasets.remove_columns(["idx", "sentence1", "sentence2"])
tokenized_datasets=tokenized_datasets.rename_column("label", "labels")


In [ ]:
tokenized_datasets=tokenized_datasets.with_format("torch")
tokenized_datasets["train"]

generation of short sample of dataset using select method

In [ ]:
small_train_dataset=tokenized_datasets["train"].select(range(100))
print(small_train_dataset)

Pre processing the Data -o preprocess the dataset, we need to convert the text to numbers the model can make sense of. This is done with TOKENIZER

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

checkpoint="bert-base-uncased"

tokenizer=AutoTokenizer.from_pretrained(checkpoint)
batch= tokenizer(
    ["My name is Sharvari", "I study at IIT Bombay"],
    ["I study at IIT Bombay", "This coffee is hot"], 
    padding=True, 
    return_tensors="pt"
)

model= AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs=model(**batch)

###Dynamic Padding


Dynamic padding means the samples in this batch should all be padded to the maximum length inside the batch. Without dynamic padding, all of the samples would have to be padded to the maximum length in the whole dataset



In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification

raw_datasets=load_dataset("glue", "mrpc")
checkpoint= "bert-base-cased"
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(examples):
  return tokenizer(
      examples["sentence1"], examples["sentence2"], padding="max_length", truncatiom=True, maximum_length=128
  )      

  tokenized_datasets=raw_datasets.map(tokenize_function, batched=True)  
  tokenied_datasets=tokenized_datasets.remove_columns[("idx", "sentence", "sentence2")]  
  tokenized_datasets=tokenized_datasets.rename("label", "labels")  
  tokenized_datasets=tokenized_datasets.with_format("torch")
                    

using dataset in std PyTorch DataLoader -----> Batches of fixed shapes but unnecessary paddding

In [ ]:
from torch.utils.data import DataLoader

train_dataloader= DataLoader(tokenized_datasets["train"], batch_size=16, shuffle=True)

for step, batch in enumerate(train_dataloader):
  print(batch["input_ids"].shape)
  if step>5:
    break

For Dynamic Padding, postpone padding in preprocessing function

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification

raw_datasets=load_dataset("glue", "mrpc")
checkpoint= "bert-base-cased"
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(examples):
  return tokenizer(
      examples["sentence1"], examples["sentence2"], truncatiom=True
  )      

  tokenized_datasets=raw_datasets.map(tokenize_function, batched=True)  
  tokenized_datasets=tokenized_datasets.remove_columns[("idx", "sentence", "sentence2")]  
  tokenized_datasets=tokenized_datasets.rename("label", "labels")  
  tokenized_datasets=tokenized_datasets.with_format("torch")     

collate function- puts together samples inside a batch 

In [ ]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

data_collator= DataCollatorWithPadding(tokenizer)
train_dataloader =DataLoader(
    tokenized_datasets["train"], batch_size=16, shuffle=True, collate_fn=data_collator
)

for step, batch in enumerate(train_dataloader):
  print(batch["input_ids"].shape)
  if step>5:
    break

Thus each batch has different but no unnecessary padding

###Fine-tuning a model with the Trainer API

1. starting

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_dataset=load_dataset("glue", "mrpc")
checkpoint="bert-base-uncased"

tokenizer=AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
  return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_dataset=raw_dataset.map(tokenize_function, batched=True)



training

In [ ]:
from transformers import TrainingArguments

training_args=TrainingArguments("test-trainer")

Defining model

In [ ]:
from transformers import AutoModelForSequenceClassification

model=AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

pass all objects into trainer

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

fine tune

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3668
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1377


Step,Training Loss


evaluation

In [ ]:
def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
tainer.train()